<a href="https://colab.research.google.com/github/ReAlex1902/stock-trend-pred/blob/main/load_text_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install feedparser
!pip install dateparser
!pip install trafilatura

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6048 sha256=93b9b1a2732cf666fbf001724fa728ad6df1e1a9ac3cf0317eba97bef14b0c22
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall:

In [8]:
from pygooglenews import GoogleNews
from bs4 import BeautifulSoup
import json
import numpy as np
import requests
from requests.models import MissingSchema
import spacy
import trafilatura
import time
import pandas as pd
import base64

In [13]:
from datetime import datetime, timedelta


def beautifulsoup_extract_text_fallback(response_content):

    '''
    This is a fallback function, so that we can always return a value for text content.
    Even for when both Trafilatura and BeautifulSoup are unable to extract the text from a
    single URL.
    '''

    # Create the beautifulsoup object:
    soup = BeautifulSoup(response_content, 'html.parser')

    # Finding the text:
    text = soup.find_all(text=True)

    # Remove unwanted tag elements:
    cleaned_text = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head',
        'input',
        'script',
        'style',]

    # Then we will loop over every item in the extract text and make sure that the beautifulsoup4 tag
    # is NOT in the blacklist
    for item in text:
        if item.parent.name not in blacklist:
            cleaned_text += '{} '.format(item)

    # Remove any tab separation and strip the text:
    cleaned_text = cleaned_text.replace('\t', '')
    return cleaned_text.strip()


def extract_text_from_single_web_page(url):

    downloaded_url = trafilatura.fetch_url(url)
    try:
        a = trafilatura.extract(downloaded_url, with_metadata=True, include_comments = False, output_format = 'json',
                            date_extraction_params={'extensive_search': True, 'original_date': True})
    except AttributeError:
        a = trafilatura.extract(downloaded_url, with_metadata=True, output_format = 'json',
                            date_extraction_params={'extensive_search': True, 'original_date': True})
    if a:
        json_output = json.loads(a)
        return json_output['text']
    else:
        try:
            headers = requests.utils.default_headers()
            headers.update(
                {
                    'User-Agent': 'My User Agent 1.0',
                }
            )
            resp = requests.get(url, headers=headers)
            # We will only extract the text from successful requests:
            if resp.status_code == 200:
                return beautifulsoup_extract_text_fallback(resp.content)
            else:
                # This line will handle for any failures in both the Trafilature and BeautifulSoup4 functions:
                return np.nan
        # Handling for any URLs that don't have the correct protocol
        except MissingSchema:
            return np.nan


def get_all_dates_between(start_date_str, end_date_str):
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

    current_date = start_date
    date_list = []

    while current_date <= end_date:
        date_list.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)

    return date_list


def get_news_between_dates(api, dates, topic):
    news_list = []

    for i in range(len(dates)-1):
        from_date = dates[i]
        to_date = dates[i+1]

        result = api.search(topic, from_=from_date, to_=to_date)
        news_list.append(result)

    return news_list


def decode_base64_code(base64_url):
    actual_url = str(base64.urlsafe_b64decode(base64_url + '=' * (4 - len(base64_url) % 4)))#[12:-13]
    actual_url = actual_url.split('http')[-1]
    actual_url = 'http' + actual_url
    actual_url = actual_url.split('?')[0]
    actual_url = actual_url.split('\\')[0]
    actual_url = actual_url.replace("'", '')

    return actual_url


# Пример использования:
gn = GoogleNews()

topic = 'Microsoft'
start_date_str = '2022-09-01'
end_date_str = '2022-10-01'

date_list = get_all_dates_between(start_date_str, end_date_str)
news_result = get_news_between_dates(gn, date_list, topic)

In [14]:
try:
    with open('used-google-links.txt') as myfile:
        used_links = myfile.readlines()
        for i in range(len(used_links)):
            used_links[i] = used_links[i].split()
    used_links_df = pd.DataFrame(used_links, columns = ['id', 'id_short', 'google-link', 'link', 'date'])
except:
    used_links_df = pd.DataFrame(columns = ['id', 'id_short', 'google-link', 'link', 'date'])


for day_idx in range(len(news_result)):
    day_news = news_result[day_idx]['entries']

    for news_idx in range(len(day_news)):
        news = day_news[news_idx]
        google_news_link = news['link']
        news_date = time.strftime('%Y-%m-%d', news['published_parsed'])
        title = news['title']
        id = news['id']
        id_short = id[:250]

        if id not in used_links_df['id'].tolist() and \
           google_news_link not in used_links_df['google-link'].tolist():

            # headers = requests.utils.default_headers()

            # headers.update(
            #     {
            #         'User-Agent': 'My User Agent 1.0',
            #     }
            # )
            # scrapy_link = f"http://api.scraperapi.com?api_key=9b0d64f7fc1613e0a0e2fe6e302b458f&url={google_news_link}"
            # news_url = requests.get(google_news_link, headers=headers).url
            # scraperapi_news_link = f"http://api.scraperapi.com?api_key=92bb15abf7a1a5ea54785dd0b45e0f8f&url={google_news_link}"
            # news_url = extract_text_from_single_web_page(scraperapi_news_link).split()[-1]
            news_url = decode_base64_code(id)
            text = extract_text_from_single_web_page(url=news_url)

            if not text is np.nan:
                print(title)
                print(google_news_link)
                print(news_url)
                print(news_date)
                print(text[:100])
                print()

                filename = str(title+'-'+news_date).replace('/', '-')
                with open(f"/content/msft-texts/{filename}.txt", 'a') as myfile:
                    myfile.write(id)
                    myfile.write("\n")
                    myfile.write(title)
                    myfile.write("\n")
                    myfile.write(text)

                with open("used-google-links.txt", "a") as myfile:
                    myfile.write(id)
                    myfile.write(" ")
                    myfile.write(id_short)
                    myfile.write(" ")
                    myfile.write(google_news_link)
                    myfile.write(" ")
                    myfile.write(news_url)
                    myfile.write(" ")
                    myfile.write(news_date)
                    myfile.write("\n")

            else:
                print(title)
                print(google_news_link)
                print(news_url)
                print(news_date)
                print('NO TEXT')
                print()

            new_row = {
                'id': id,
                'id_short': id_short,
                'google-link': google_news_link,
                'link': news_url,
                'date': news_date
            }
            used_links_df = pd.concat([used_links_df, pd.Series(new_row).to_frame().T], ignore_index = True)

        # break
    # break

Gaming for everyone, everywhere: our view on the Activision Blizzard acquisition - Microsoft On the Issues - Microsoft
https://news.google.com/rss/articles/CBMiUGh0dHBzOi8vYmxvZ3MubWljcm9zb2Z0LmNvbS9vbi10aGUtaXNzdWVzLzIwMjIvMDkvMDEvZ2FtaW5nLWV2ZXJ5b25lLWV2ZXJ5d2hlcmUv0gEA?oc=5
https://blogs.microsoft.com/on-the-issues/2022/09/01/gaming-everyone-everywhere/
2022-09-01
Game developers around the world are creating innovative and groundbreaking games. We think that we 

Microsoft's Activision Blizzard deal gets global scrutiny - The Associated Press
https://news.google.com/rss/articles/CBMiXmh0dHBzOi8vYXBuZXdzLmNvbS9hcnRpY2xlL2JpZGVuLXRlY2hub2xvZ3ktbmV3LXplYWxhbmQtZ2FtZXMtYWMxYzNiODEyNjdmNDBmMWQyYmUyNDA2YTc2NGMxYmbSAQA?oc=5
https://apnews.com/article/biden-technology-new-zealand-games-ac1c3b81267f40f1d2be2406a764c1bf
2022-09-01
Microsoft’s Activision Blizzard deal gets global scrutiny
Microsoft’s plan to buy video game giant A



ERROR:trafilatura.downloads:not a 200 response: 401 for URL https://www.reuters.com/markets/deals/uk-watchdog-further-probe-microsofts-687-billion-activision-deal-2022-09-01/


Microsoft will keep Call of Duty on PlayStation for 'several more years' beyond existing deal - The Verge
https://news.google.com/rss/articles/CBMieGh0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDIyLzkvMi8yMzMzNDYxOS9taWNyb3NvZnQtc29ueS1jYWxsLW9mLWR1dHktcGxheXN0YXRpb24tbGV0dGVyLWNvbW1pdG1lbnQtYWN0aXZpc2lvbi1ibGl6emFyZNIBAA?oc=5
https://www.theverge.com/2022/9/2/23334619/microsoft-sony-call-of-duty-playstation-letter-commitment-activision-blizzard
2022-09-02
Microsoft is committing to keeping Call of Duty on PlayStation for “several more years” beyond the e

Microsoft's $69 bln Activision deal could harm competition, UK says - Reuters.com
https://news.google.com/rss/articles/CBMicmh0dHBzOi8vd3d3LnJldXRlcnMuY29tL21hcmtldHMvZGVhbHMvdWstd2F0Y2hkb2ctZnVydGhlci1wcm9iZS1taWNyb3NvZnRzLTY4Ny1iaWxsaW9uLWFjdGl2aXNpb24tZGVhbC0yMDIyLTA5LTAxL9IBAA?oc=5
https://www.reuters.com/markets/deals/uk-watchdog-further-probe-microsofts-687-billion-activision-deal-2022-09-01/
2022-09-01
NO TEXT

AI is helping vulnerable 

ERROR:trafilatura.downloads:download error: https://www.nasdaq.com/articles/how-microsoft-is-building-its-security-business HTTPSConnectionPool(host='www.nasdaq.com', port=443): Max retries exceeded with url: /articles/how-microsoft-is-building-its-security-business (Caused by ReadTimeoutError("HTTPSConnectionPool(host='www.nasdaq.com', port=443): Read timed out. (read timeout=30)"))
<ipython-input-13-16fea7a00a3e>:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text = soup.find_all(text=True)


How Microsoft Is Building Its Security Business - Nasdaq
https://news.google.com/rss/articles/CBMiT2h0dHBzOi8vd3d3Lm5hc2RhcS5jb20vYXJ0aWNsZXMvaG93LW1pY3Jvc29mdC1pcy1idWlsZGluZy1pdHMtc2VjdXJpdHktYnVzaW5lc3PSAQA?oc=5
https://www.nasdaq.com/articles/how-microsoft-is-building-its-security-business
2022-09-02
How Microsoft Is Building Its Security Business | Nasdaq 
 
 
 
 
 
 
 
 
 
 
 
   
 
 
 
 
 
 Skip 

Microsoft’s Activision Blizzard Acquisition Under More Scrutiny As U.K. Launches ‘In-Depth’ Antitrust Probe - Forbes
https://news.google.com/rss/articles/CBMinAFodHRwczovL3d3dy5mb3JiZXMuY29tL3NpdGVzL3NpbGFkaXR5YXJheS8yMDIyLzA5LzAxL21pY3Jvc29mdHMtYWN0aXZpc2lvbi1ibGl6emFyZC1hY3F1aXNpdGlvbi11bmRlci1tb3JlLXNjcnV0aW55LWFzLXVrLWxhdW5jaGVzLWluLWRlcHRoLWFudGl0cnVzdC1wcm9iZS_SAaABaHR0cHM6Ly93d3cuZm9yYmVzLmNvbS9zaXRlcy9zaWxhZGl0eWFyYXkvMjAyMi8wOS8wMS9taWNyb3NvZnRzLWFjdGl2aXNpb24tYmxpenphcmQtYWNxdWlzaXRpb24tdW5kZXItbW9yZS1zY3J1dGlueS1hcy11ay1sYXVuY2hlcy1pbi1kZXB0aC1hbnRpdHJ1c3QtcHJvYmUvYW1wLw?oc=

...

Microsoft Edge starts testing a new dedicated ‘wallet’ page to help users manage saved cards easily - Times of India
https://news.google.com/rss/articles/CBMiqwFodHRwczovL3RpbWVzb2ZpbmRpYS5pbmRpYXRpbWVzLmNvbS9nYWRnZXRzLW5ld3MvbWljcm9zb2Z0LWVkZ2Utc3RhcnRzLXRlc3RpbmctYS1uZXctZGVkaWNhdGVkLXdhbGxldC1wYWdlLXRvLWhlbHAtdXNlcnMtbWFuYWdlLXNhdmVkLWNhcmRzLWVhc2lseS9hcnRpY2xlc2hvdy85NDE5OTYxMS5jbXPSAa8BaHR0cHM6Ly90aW1lc29maW5kaWEuaW5kaWF0aW1lcy5jb20vZ2FkZ2V0cy1uZXdzL21pY3Jvc29mdC1lZGdlLXN0YXJ0cy10ZXN0aW5nLWEtbmV3LWRlZGljYXRlZC13YWxsZXQtcGFnZS10by1oZWxwLXVzZXJzLW1hbmFnZS1zYXZlZC1jYXJkcy1lYXNpbHkvYW1wX2FydGljbGVzaG93Lzk0MTk5NjExLmNtcw?oc=5
https://timesofindia.indiatimes.com/gadgets-news/microsoft-edge-starts-testing-a-new-dedicated-wallet-page-to-help-users-manage-saved-cards-easily/amp_articleshow/94199611.cms
2022-09-14
This story is from September 14, 2022
Microsoft Edge starts testing a new dedicated ‘wallet’ page to



<ipython-input-13-16fea7a00a3e>:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text = soup.find_all(text=True)


Lumen Technologies names former Microsoft exec as new CEO - Denver Business Journal - The Business Journals
https://news.google.com/rss/articles/CBMiWGh0dHBzOi8vd3d3LmJpempvdXJuYWxzLmNvbS9kZW52ZXIvbmV3cy8yMDIyLzA5LzE0L2x1bWVuLXRlY2hub2xvZ2llcy1uYW1lcy1uZXctY2VvLmh0bWzSAQA?oc=5
https://www.bizjournals.com/denver/news/2022/09/14/lumen-technologies-names-new-ceo.html
2022-09-14
Request unsuccessful. Incapsula incident ID: 1427000850590411423-441999070254536005

10 Ways to Fix the Windows Search Bar Not Showing or Working on Windows 11 - MUO - MakeUseOf
https://news.google.com/rss/articles/CBMiR2h0dHBzOi8vd3d3Lm1ha2V1c2VvZi5jb20vd2luZG93cy0xMS1zZWFyY2gtYmFyLW5vdC1zaG93aW5nLXdvcmtpbmctb24v0gEA?oc=5
https://www.makeuseof.com/windows-11-search-bar-not-showing-working-on/
2022-09-14
Did you start your PC today only to find your Windows search bar missing or not working? The Windows

HCL Tech Lays Off 300 Employees Globally Working On Microsoft Projects: Report - Outlook India
https://news.goog

ERROR:trafilatura.downloads:download error: https://www.civilserviceworld.com/professions/article/dwp-signs-170k-deal-for-microsoft-teams-training HTTPSConnectionPool(host='www.civilserviceworld.com', port=443): Max retries exceeded with url: /professions/article/dwp-signs-170k-deal-for-microsoft-teams-training (Caused by ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')))


ConnectionError: ignored

In [15]:
used_links_df

,id,id_short,google-link,link,date
0,CBMiigFodHRwczovL25ld3MubWljcm9zb2Z0LmNvbS9lbi...,CBMiigFodHRwczovL25ld3MubWljcm9zb2Z0LmNvbS9lbi...,https://news.google.com/rss/articles/CBMiigFod...,https://news.microsoft.com/en-in/microsoft-cel...,2023-01-02
1,CBMiigFodHRwczovL3d3dy5mb3JiZXMuY29tL3NpdGVzL2...,CBMiigFodHRwczovL3d3dy5mb3JiZXMuY29tL3NpdGVzL2...,https://news.google.com/rss/articles/CBMiigFod...,https://www.forbes.com/sites/janakirammsv/2023...,2023-01-01
2,CBMidWh0dHBzOi8vd2FzaGluZ3RvbmV4ZWMuY29tLzIwMj...,CBMidWh0dHBzOi8vd2FzaGluZ3RvbmV4ZWMuY29tLzIwMj...,https://news.google.com/rss/articles/CBMidWh0d...,https://washingtonexec.com/2023/01/top-chief-t...,2023-01-02
3,CBMiZ2h0dHBzOi8vd3d3LmVuZ2FkZ2V0LmNvbS9wcmUtdH...,CBMiZ2h0dHBzOi8vd3d3LmVuZ2FkZ2V0LmNvbS9wcmUtdH...,https://news.google.com/rss/articles/CBMiZ2h0d...,https://www.engadget.com/pre-trial-hearing-bet...,2023-01-01
4,CBMiTGh0dHBzOi8vd3d3Lm1ha2V1c2VvZi5jb20vd2luZG...,CBMiTGh0dHBzOi8vd3d3Lm1ha2V1c2VvZi5jb20vd2luZG...,https://news.google.com/rss/articles/CBMiTGh0d...,https://www.makeuseof.com/windows-switch-local...,2023-01-01
...,...,...,...,...,...
11405,CBMiqwFodHRwczovL3RpbWVzb2ZpbmRpYS5pbmRpYXRpbW...,CBMiqwFodHRwczovL3RpbWVzb2ZpbmRpYS5pbmRpYXRpbW...,https://news.google.com/rss/articles/CBMiqwFod...,https://timesofindia.indiatimes.com/gadgets-ne...,2022-09-14
11406,CBMiWGh0dHBzOi8vd3d3LmJpempvdXJuYWxzLmNvbS9kZW...,CBMiWGh0dHBzOi8vd3d3LmJpempvdXJuYWxzLmNvbS9kZW...,https://news.google.com/rss/articles/CBMiWGh0d...,https://www.bizjournals.com/denver/news/2022/0...,2022-09-14
11407,CBMiR2h0dHBzOi8vd3d3Lm1ha2V1c2VvZi5jb20vd2luZG...,CBMiR2h0dHBzOi8vd3d3Lm1ha2V1c2VvZi5jb20vd2luZG...,https://news.google.com/rss/articles/CBMiR2h0d...,https://www.makeuseof.com/windows-11-search-ba...,2022-09-14
11408,CBMif2h0dHBzOi8vd3d3Lm91dGxvb2tpbmRpYS5jb20vYn...,CBMif2h0dHBzOi8vd3d3Lm91dGxvb2tpbmRpYS5jb20vYn...,https://news.google.com/rss/articles/CBMif2h0d...,https://www.outlookindia.com/business/hcl-tech...,2022-09-14


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [28]:
!cp -r /content/msft-texts /content/gdrive/Shareddrives/gdrive/datasets/thesis
!cp /content/used-google-links.txt /content/gdrive/Shareddrives/gdrive/datasets/thesis

In [2]:
!cp -r /content/gdrive/Shareddrives/gdrive/datasets/thesis/msft-texts /content
!cp /content/gdrive/Shareddrives/gdrive/datasets/thesis/used-google-links.txt /content

In [3]:
import pandas as pd

try:
    with open('used-google-links.txt') as myfile:
        used_links = myfile.readlines()
        for i in range(len(used_links)):
            used_links[i] = used_links[i].split()
    used_links_df = pd.DataFrame(used_links, columns = ['id', 'id_short', 'google-link', 'link', 'date'])
except:
    used_links_df = pd.DataFrame(columns = ['id', 'id_short', 'google-link', 'link', 'date'])

In [26]:
used_links_df['date'].apply(lambda x:x[:-3]).value_counts()

2023-01    1065
2023-03    1065
2022-11    1041
2023-05    1024
2022-10     987
2023-09     984
2023-07     953
2023-02     885
2023-06     865
2023-08     786
2022-12     785
2023-04     514
2022-09     456
Name: date, dtype: int64

In [27]:
import os
pd.DataFrame(os.listdir('/content/msft-texts'))[0].str.split('-').str[-3:-1].value_counts()

[2023, 03]    1065
[2023, 01]    1064
[2022, 11]    1041
[2023, 05]    1024
[2022, 10]     987
[2023, 09]     984
[2023, 07]     953
[2023, 02]     885
[2023, 06]     865
[2023, 08]     786
[2022, 12]     785
[2023, 04]     514
[2022, 09]     444
[]               1
Name: 0, dtype: int64